<a href="https://colab.research.google.com/github/smara97/Machine-Learning-Projects/blob/master/Generate%20Text%20Language%20Model%20/RNN_LM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd
import torch
from torch import nn
import torch.nn.functional as F
from io import open
from google.colab import files

In [2]:
up=files.upload()

Saving anna.txt to anna.txt


In [0]:
lines=[]
with open ("/content/anna.txt",'r',encoding="utf8") as f:
    lines=f.read()

In [4]:
chars=tuple(set(lines))
print(chars)

('p', 's', '_', 'j', ';', '*', 'x', 'm', 'k', 'D', '$', 'n', '/', 'O', 'P', '2', 'X', 'q', 'e', ':', '.', "'", ' ', 'H', 'I', '(', '-', '@', '7', 'Y', 'K', '?', 'A', '4', 'u', '3', 'R', 'V', 'h', 'r', 'y', 'a', '1', '"', 'J', 'o', ')', 'L', 'g', 'f', 'w', 't', 'c', 'Q', '&', 'v', 'M', 'z', '8', 'd', '0', 'U', '9', '`', ',', 'b', 'i', '!', 'G', 'Z', '6', '%', '5', 'l', 'E', 'B', 'T', 'S', 'F', '\n', 'W', 'C', 'N')


In [5]:
int2char=dict(enumerate(chars))
char2int={ch:ii for ii,ch in int2char.items()}
print(int2char)
print(char2int)

{0: 'p', 1: 's', 2: '_', 3: 'j', 4: ';', 5: '*', 6: 'x', 7: 'm', 8: 'k', 9: 'D', 10: '$', 11: 'n', 12: '/', 13: 'O', 14: 'P', 15: '2', 16: 'X', 17: 'q', 18: 'e', 19: ':', 20: '.', 21: "'", 22: ' ', 23: 'H', 24: 'I', 25: '(', 26: '-', 27: '@', 28: '7', 29: 'Y', 30: 'K', 31: '?', 32: 'A', 33: '4', 34: 'u', 35: '3', 36: 'R', 37: 'V', 38: 'h', 39: 'r', 40: 'y', 41: 'a', 42: '1', 43: '"', 44: 'J', 45: 'o', 46: ')', 47: 'L', 48: 'g', 49: 'f', 50: 'w', 51: 't', 52: 'c', 53: 'Q', 54: '&', 55: 'v', 56: 'M', 57: 'z', 58: '8', 59: 'd', 60: '0', 61: 'U', 62: '9', 63: '`', 64: ',', 65: 'b', 66: 'i', 67: '!', 68: 'G', 69: 'Z', 70: '6', 71: '%', 72: '5', 73: 'l', 74: 'E', 75: 'B', 76: 'T', 77: 'S', 78: 'F', 79: '\n', 80: 'W', 81: 'C', 82: 'N'}
{'p': 0, 's': 1, '_': 2, 'j': 3, ';': 4, '*': 5, 'x': 6, 'm': 7, 'k': 8, 'D': 9, '$': 10, 'n': 11, '/': 12, 'O': 13, 'P': 14, '2': 15, 'X': 16, 'q': 17, 'e': 18, ':': 19, '.': 20, "'": 21, ' ': 22, 'H': 23, 'I': 24, '(': 25, '-': 26, '@': 27, '7': 28, 'Y': 29, 

In [0]:
encode=np.array([char2int[ch] for ch in lines])

In [0]:
def one_hot_encode(arr,labels):
    
    one_hot=np.zeros((np.multiply(*arr.shape),labels),dtype=np.float32)
    one_hot[np.arange(one_hot.shape[0]),arr.flatten()]=1
    one_hot=one_hot.reshape((*arr.shape,labels))
    
    return one_hot

In [0]:
def get_batches(arr, batch_size, seq_length):

    batch_size_total = batch_size * seq_length

    n_batches = len(arr)//batch_size_total
    
    arr = arr[:n_batches * batch_size_total]
    
    arr = arr.reshape((batch_size, -1))
    
    for n in range(0, arr.shape[1], seq_length):
        x = arr[:, n:n+seq_length]
        y = np.zeros_like(x)
        try:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, n+seq_length]
        except IndexError:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, 0]
        yield x, y

In [0]:
class RNN(nn.Module):
    def __init__(self,tokens,n_hidden=265,n_layers=2,drop_prop=0.5,lr=0.001):
        super().__init__()
        self.drop_prop=drop_prop
        self.lr=lr
        self.n_layers=n_layers
        self.n_hidden=n_hidden
        
        self.chars=tokens
        self.int2char=dict(enumerate(self.chars))
        self.char2int={ch:ii for ii,ch in self.int2char.items()}
        
        self.lstm=nn.LSTM(len(self.chars),n_hidden,n_layers,dropout=drop_prop,batch_first=True)
        self.dropout=nn.Dropout(drop_prop)
        self.fc=nn.Linear(n_hidden,len(chars))
        
    def forward(self,x,hidden):
        r_output,hidden=self.lstm(x,hidden)
        out=self.dropout(r_output)
        out=out.contiguous().view(-1,self.n_hidden)
        out=self.fc(out)
        
        return out,hidden
    
    def init_hidden(self,batch_size):
        weight=next(self.parameters()).data.cuda()
        hidden=(weight.new(self.n_layers,batch_size,self.n_hidden).zero_().cuda(),
                weight.new(self.n_layers,batch_size,self.n_hidden).zero_().cuda())
        
        
        return hidden
        
        
        
        

In [54]:
net=RNN(chars,512,2)
net.cuda()
print(net)

RNN(
  (lstm): LSTM(83, 512, num_layers=2, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc): Linear(in_features=512, out_features=83, bias=True)
)


In [0]:
def train(net, data, epochs=10, batch_size=10, seq_length=50, lr=0.001, clip=5, val_frac=0.1, print_every=10):
    
    net.train()
    opt=torch.optim.Adam(net.parameters(),lr=lr)
    crit=nn.CrossEntropyLoss()
    val_idx = int(len(data)*(1-val_frac))
    data, val_data = data[:val_idx], data[val_idx:]
    
    counter=0
    n_chars=len(net.chars)
    for e in range(epochs):
        
        h=net.init_hidden(batch_size)
        
        for x,y in get_batches(data,batch_size,seq_length):
            counter+=1
        
            x=one_hot_encode(x,n_chars)
            inputs,targets=torch.from_numpy(x),torch.from_numpy(y)
            inputs,targets=inputs.cuda(),targets.cuda()
            h=tuple([each.data for each in h])
            
            net.zero_grad()
            out,h=net(inputs,h)
            
            loss=crit(out,targets.view(batch_size*seq_length).long())
            loss.backward()
            
            nn.utils.clip_grad_norm(net.parameters(),clip)
            opt.step()
        val_h=net.init_hidden(batch_size)
        val_losses=[]
        net.eval()
                
        for x,y in get_batches(val_data,batch_size,seq_length):
          
          
          x=one_hot_encode(x,n_chars)
                    
          x,y=torch.from_numpy(x),torch.from_numpy(y)
          
                    
          val_h=tuple([each.data for each in val_h])
          inputs,targets=x,y
          inputs,targets=inputs.cuda(),targets.cuda()
          out,val_h=net(inputs,val_h)
          val_loss=crit(out,targets.view(batch_size*seq_length).long())
          val_losses.append(val_loss.item())
          net.train()
                
        print("Epoch: {}/{}...".format(e+1, epochs),
              "Loss: {:.4f}...".format(loss.item()),
              "Val Loss: {:.4f}".format(np.mean(val_losses)))
                
                    

In [56]:
batch_size = 128
seq_length = 100
n_epochs = 20 
train(net, encode, epochs=n_epochs, batch_size=batch_size, seq_length=seq_length, lr=0.001, print_every=10)
model_name = 'rnn_20_epoch.net'

checkpoint = {'n_hidden': net.n_hidden,
              'n_layers': net.n_layers,
              'state_dict': net.state_dict(),
              'tokens': net.chars}

with open(model_name, 'wb') as f:
  torch.save(checkpoint, f)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:29: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_grad_norm_.


Epoch: 1/20... Loss: 2.8070... Val Loss: 2.8055
Epoch: 2/20... Loss: 2.2652... Val Loss: 2.3013
Epoch: 3/20... Loss: 2.0332... Val Loss: 2.0726
Epoch: 4/20... Loss: 1.8653... Val Loss: 1.9232
Epoch: 5/20... Loss: 1.7516... Val Loss: 1.8095
Epoch: 6/20... Loss: 1.6593... Val Loss: 1.7383
Epoch: 7/20... Loss: 1.5960... Val Loss: 1.6782
Epoch: 8/20... Loss: 1.5573... Val Loss: 1.6320
Epoch: 9/20... Loss: 1.5091... Val Loss: 1.5921
Epoch: 10/20... Loss: 1.4698... Val Loss: 1.5576
Epoch: 11/20... Loss: 1.4395... Val Loss: 1.5378
Epoch: 12/20... Loss: 1.4128... Val Loss: 1.5144
Epoch: 13/20... Loss: 1.3969... Val Loss: 1.4948
Epoch: 14/20... Loss: 1.3724... Val Loss: 1.4775
Epoch: 15/20... Loss: 1.3556... Val Loss: 1.4619
Epoch: 16/20... Loss: 1.3299... Val Loss: 1.4525
Epoch: 17/20... Loss: 1.3257... Val Loss: 1.4444
Epoch: 18/20... Loss: 1.3147... Val Loss: 1.4346
Epoch: 19/20... Loss: 1.3033... Val Loss: 1.4253
Epoch: 20/20... Loss: 1.2934... Val Loss: 1.4134


In [0]:
def predict(net ,char,h=None,top_k=None):
  
  x = np.array([[net.char2int[char]]])
  x = one_hot_encode(x, len(net.chars))
  inputs=torch.from_numpy(x).cuda()
  
  h=tuple([each.data for each in h])
  out,h=net(inputs,h)
  
  p=F.softmax(out,dim=1).data
  
  p=p.cpu()
  
  if top_k is None:
    top_ch = np.arange(len(net.chars))
  else:
    p, top_ch = p.topk(top_k)
    top_ch = top_ch.numpy().squeeze()    
  p = p.numpy().squeeze()
  char = np.random.choice(top_ch, p=p/p.sum())
  return net.int2char[char],h

In [0]:
def sample(net, size, prime='The', top_k=None):
  
        
  net.cuda()
    
  net.eval() 

  chars = [ch for ch in prime]
  h = net.init_hidden(1)
  for ch in prime:
    char, h = predict(net, ch, h, top_k=top_k)
  chars.append(char)
    
  for ii in range(size):
    char, h = predict(net, chars[-1], h, top_k=top_k)
    chars.append(char)

  return ''.join(chars)

In [62]:
print(sample(net, 1000, prime='Ahmed', top_k=5))

Ahmed the
face in the
courcity of his face to tell them
in them.

The strong
study and the papard hands, a capital ask and southing of the
children. This was not still at once to go to his foreer. Seeing that
there was to saw it. She saw her. And all that had already to be sole to the position. Anna had always talked to him in a setter
of man, and an annoyances he could
nater her secretary. She did
not be dispuce, the same
time as they were some sister than ever as it herself went into her arm and took the parts of
souldence, and
to the motion, he saw.

"You don't stret the letter."

"Ah!" said Levin, with silent as the marsh. And answered the marshal with his boy. He smoke with happiness, and without taking a people and then the mire as a pealit carm, so interrupted her for the
pretemptoon and say with the carriage for this, almost all her hands the
decones of the money of the strange had stopped his faculty, as
she was streint to the carriage, and they
saw him that, as though he care